In [2]:
import scanpy as sc
import pandas as pd
import pickle
import time
import anndata
import sklearn.metrics
%matplotlib notebook
%pylab

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


In [4]:
adata = sc.read_10x_mtx('matrix_pbmc',var_names='gene_symbols',cache=True)  
sc.settings.set_figure_params(dpi=80)
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
mito_genes = adata.var_names.str.startswith('MT-') 
adata.obs['percent_mito'] = np.sum(adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1 
adata.obs['n_counts'] = adata.X.sum(axis=1).A1
adata = adata[adata.obs['n_genes'] < 2500, :]
adata = adata[adata.obs['percent_mito'] < 0.05, :]
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)
adata.raw = adata
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var['highly_variable']]
sc.pp.regress_out(adata, ['n_counts', 'percent_mito'])
sc.pp.scale(adata, max_value=10) #i geni con una varianza più alta di 10 gli viene attribuita varianza uguale a 10
sc.tl.pca(adata, svd_solver='arpack')
adata.write('mi_pbmc/adata_base.h5ad')

Trying to set attribute `.obs` of view, making a copy.


In [5]:
adata=sc.read('mi_pbmc/adata_base.h5ad')

In [6]:
nei=[5,10,20,50,100]
adatone=[]
for x in nei:
    adata=sc.read('mi_pbmc/adata_base.h5ad')
    sc.pp.neighbors(adata, n_neighbors=x, n_pcs=40)
    adatone.append(adata)
adatone[0].write('mi_pbmc/adata_5.h5ad')
adatone[1].write('mi_pbmc/adata_10.h5ad')
adatone[2].write('mi_pbmc/adata_20.h5ad')
adatone[3].write('mi_pbmc/adata_50.h5ad')
adatone[4].write('mi_pbmc/adata_100.h5ad')

In [7]:
adata5=sc.read('mi_pbmc/adata_5.h5ad')
adata10=sc.read('mi_pbmc/adata_10.h5ad')
adata20=sc.read('mi_pbmc/adata_20.h5ad')
adata50=sc.read('mi_pbmc/adata_50.h5ad')
adata100=sc.read('mi_pbmc/adata_100.h5ad')

In [8]:
sc.tl.umap(adata5)
sc.tl.umap(adata10) 
sc.tl.umap(adata20) 
sc.tl.umap(adata50)
sc.tl.umap(adata100) 

# 1) leiden in range
### 1.1) leiden unweighted

In [9]:
resolutions = np.arange(0, 1, 0.005)

In [10]:
leiden_clusters = pd.DataFrame(index=adata5.obs_names, columns=resolutions)
for r in resolutions:
    sc.tl.leiden(adata5, resolution=r, use_weights=False)
    leiden_clusters.loc[:, r] = adata5.obs.leiden
leiden_clusters.to_csv('mi_pbmc/dropseq_leiden_clusters_5_unweighted.csv')

In [11]:
leiden_clusters = pd.DataFrame(index=adata10.obs_names, columns=resolutions)
for r in resolutions:
    sc.tl.leiden(adata10, resolution=r, use_weights=False)
    leiden_clusters.loc[:, r] = adata10.obs.leiden
leiden_clusters.to_csv('mi_pbmc/dropseq_leiden_clusters_10_unweighted.csv')

In [12]:
leiden_clusters = pd.DataFrame(index=adata20.obs_names, columns=resolutions)
for r in resolutions:
    sc.tl.leiden(adata20, resolution=r, use_weights=False)
    leiden_clusters.loc[:, r] = adata20.obs.leiden
leiden_clusters.to_csv('mi_pbmc/dropseq_leiden_clusters_20_unweighted.csv')

In [13]:
leiden_clusters = pd.DataFrame(index=adata50.obs_names, columns=resolutions)
for r in resolutions:
    sc.tl.leiden(adata50, resolution=r, use_weights=False)
    leiden_clusters.loc[:, r] = adata50.obs.leiden
leiden_clusters.to_csv('mi_pbmc/dropseq_leiden_clusters_50_unweighted.csv')

In [14]:
leiden_clusters = pd.DataFrame(index=adata100.obs_names, columns=resolutions)
for r in resolutions:
    sc.tl.leiden(adata100, resolution=r, use_weights=False)
    leiden_clusters.loc[:, r] = adata100.obs.leiden
leiden_clusters.to_csv('mi_pbmc/dropseq_leiden_clusters_100_unweighted.csv')

In [15]:
unw_leiden_clusters5=pd.read_csv('mi_pbmc/dropseq_leiden_clusters_5_unweighted.csv')
unw_leiden_clusters10=pd.read_csv('mi_pbmc/dropseq_leiden_clusters_10_unweighted.csv')
unw_leiden_clusters20=pd.read_csv('mi_pbmc/dropseq_leiden_clusters_20_unweighted.csv')
unw_leiden_clusters50=pd.read_csv('mi_pbmc/dropseq_leiden_clusters_50_unweighted.csv')
unw_leiden_clusters100=pd.read_csv('mi_pbmc/dropseq_leiden_clusters_100_unweighted.csv')

### 1.2) leiden weighted

In [16]:
adata5=sc.read('mi_pbmc/adata_5.h5ad')
adata10=sc.read('mi_pbmc/adata_10.h5ad')
adata20=sc.read('mi_pbmc/adata_20.h5ad')
adata50=sc.read('mi_pbmc/adata_50.h5ad')
adata100=sc.read('mi_pbmc/adata_100.h5ad')

In [17]:
leiden_clusters = pd.DataFrame(index=adata5.obs_names, columns=resolutions)
for r in resolutions:
    sc.tl.leiden(adata5, resolution=r, use_weights=True)
    leiden_clusters.loc[:, r] = adata5.obs.leiden
leiden_clusters.to_csv('mi_pbmc/dropseq_leiden_clusters_5_weighted.csv')

In [18]:
leiden_clusters = pd.DataFrame(index=adata10.obs_names, columns=resolutions)
for r in resolutions:
    sc.tl.leiden(adata10, resolution=r, use_weights=True)
    leiden_clusters.loc[:, r] = adata10.obs.leiden
leiden_clusters.to_csv('mi_pbmc/dropseq_leiden_clusters_10_weighted.csv')

In [19]:
leiden_clusters = pd.DataFrame(index=adata20.obs_names, columns=resolutions)
for r in resolutions:
    sc.tl.leiden(adata20, resolution=r, use_weights=True)
    leiden_clusters.loc[:, r] = adata20.obs.leiden
leiden_clusters.to_csv('mi_pbmc/dropseq_leiden_clusters_20_weighted.csv')

In [20]:
leiden_clusters = pd.DataFrame(index=adata50.obs_names, columns=resolutions)
for r in resolutions:
    sc.tl.leiden(adata50, resolution=r, use_weights=True)
    leiden_clusters.loc[:, r] = adata50.obs.leiden
leiden_clusters.to_csv('mi_pbmc/dropseq_leiden_clusters_50_weighted.csv')

In [21]:
leiden_clusters = pd.DataFrame(index=adata100.obs_names, columns=resolutions)
for r in resolutions:
    sc.tl.leiden(adata100, resolution=r, use_weights=True)
    leiden_clusters.loc[:, r] = adata100.obs.leiden
leiden_clusters.to_csv('mi_pbmc/dropseq_leiden_clusters_100_weighted.csv')

In [22]:
unw_leiden_clusters5=pd.read_csv('mi_pbmc/dropseq_leiden_clusters_5_weighted.csv')
unw_leiden_clusters10=pd.read_csv('mi_pbmc/dropseq_leiden_clusters_10_weighted.csv')
unw_leiden_clusters20=pd.read_csv('mi_pbmc/dropseq_leiden_clusters_20_weighted.csv')
unw_leiden_clusters50=pd.read_csv('mi_pbmc/dropseq_leiden_clusters_50_weighted.csv')
unw_leiden_clusters100=pd.read_csv('mi_pbmc/dropseq_leiden_clusters_100_weighted.csv')

# 2) nsbm
### 2.1) nsbm unweighted

In [ ]:
adata5=sc.read('mi_pbmc/adata_5.h5ad')
adata10=sc.read('mi_pbmc/adata_10.h5ad')
adata20=sc.read('mi_pbmc/adata_20.h5ad')
adata50=sc.read('mi_pbmc/adata_50.h5ad')
adata100=sc.read('mi_pbmc/adata_100.h5ad')

In [ ]:
a=time.time()
sc.tl.nsbm(adata5, collect_marginals=False)
b=time.time()
sc.tl.nsbm(adata10, collect_marginals=False)
c=time.time()
sc.tl.nsbm(adata20, collect_marginals=False)
d=time.time()
sc.tl.nsbm(adata50, collect_marginals=False)
e=time.time()
sc.tl.nsbm(adata100, collect_marginals=False)
f=time.time()
aa=b-a
bb=c-b
cc=d-c
dd=e-d
ee=f-e
print(aa,bb,cc,dd,ee)
adata5.write('mi_pbmc/adata_5_unweighted.h5ad')
adata10.write('mi_pbmc/adata_10_unweighted.h5ad')
adata20.write('mi_pbmc/adata_20_unweighted.h5ad')
adata50.write('mi_pbmc/adata_50_unweighted.h5ad')
adata100.write('mi_pbmc/adata_100_unweighted.h5ad')

In [ ]:
unw_adata5=sc.read('mi_pbmc/adata_5_unweighted.h5ad')
unw_adata10=sc.read('mi_pbmc/adata_10_unweighted.h5ad')
unw_adata20=sc.read('mi_pbmc/adata_20_unweighted.h5ad')
unw_adata50=sc.read('mi_pbmc/adata_50_unweighted.h5ad')
unw_adata100=sc.read('mi_pbmc/adata_100_unweighted.h5ad')

### 2.2) nsbm weighted

In [ ]:
adata5=sc.read('mi_pbmc/adata_5.h5ad')
adata10=sc.read('mi_pbmc/adata_10.h5ad')
adata20=sc.read('mi_pbmc/adata_20.h5ad')
adata50=sc.read('mi_pbmc/adata_50.h5ad')
adata100=sc.read('mi_pbmc/adata_100.h5ad')

In [ ]:
a=time.time()
sc.tl.nsbm(adata5, collect_marginals=False, use_weights=True)
b=time.time()
sc.tl.nsbm(adata10, collect_marginals=False, use_weights=True)
c=time.time()
sc.tl.nsbm(adata20, collect_marginals=False, use_weights=True)
d=time.time()
sc.tl.nsbm(adata50, collect_marginals=False, use_weights=True)
e=time.time()
sc.tl.nsbm(adata100, collect_marginals=False, use_weights=True)
f=time.time()
aa=b-a
bb=c-b
cc=d-c
dd=e-d
ee=f-e
print(aa,bb,cc,dd,ee)
adata5.write('mi_pbmc/adata_5_weighted.h5ad')
adata10.write('mi_pbmc/adata_10_weighted.h5ad')
adata20.write('mi_pbmc/adata_20_weighted.h5ad')
adata50.write('mi_pbmc/adata_50_weighted.h5ad')
adata100.write('mi_pbmc/adata_100_weighted.h5ad')

In [ ]:
unw_adata5=sc.read('mi_pbmc/adata_5_weighted.h5ad')
unw_adata10=sc.read('mi_pbmc/adata_10_weighted.h5ad')
unw_adata20=sc.read('mi_pbmc/adata_20_weighted.h5ad')
unw_adata50=sc.read('mi_pbmc/adata_50_weighted.h5ad')
unw_adata100=sc.read('mi_pbmc/adata_100_weighted.h5ad')

# 3) MI comparison
### 3.1) nsbm unweighted vs leiden unweighted

### 3.2) nsbm weighted vs leiden weighted

### 3.3) nsbm weighted vs leiden unweighted

### 3.4) nsbm unweighted vs leiden weighted